In [1]:
import pinocchio as pin
import numpy as np
import time
import scipy
from example_robot_data import load

## visualise the robot
from pinocchio.visualize import MeshcatVisualizer

## visualise the polytope and the ellipsoid
import meshcat.geometry as g 

# import pycapacity 
import pycapacity as pycap

In [2]:
# use reachy's right arm with only 4 joints for now as they are the contribute 
# to its force capacity the most
urdf_path = 'reachy.urdf'
robot = pin.RobotWrapper.BuildFromURDF(urdf_path)
model, data = robot.model, robot.data
# lock the other joints
tolock = [
    'l_shoulder_pitch',
    'l_shoulder_roll',
    'l_elbow_yaw',
    'l_elbow_pitch',
    'l_wrist_roll',
    'l_wrist_pitch',
    'l_wrist_yaw',
    'l_hand_finger',
    'l_hand_finger_mimic',
    # 'r_elbow_yaw',
    # 'r_elbow_pitch'
    # 'r_wrist_roll',
    # 'r_wrist_pitch',,
    # 'r_wrist_yaw',
    'r_hand_finger',
    'r_hand_finger_mimic',
    'neck_roll',
    'neck_pitch',
    'neck_yaw',
]

# Get the ID of all existing joints
jointsToLockIDs = []
for jn in tolock:
    if model.existJointName(jn):
        jointsToLockIDs.append(model.getJointId(jn))
robot.model = pin.buildReducedModel(model, jointsToLockIDs, np.zeros(21))
model, data = robot.model, robot.data

In [6]:
robot.data = robot.model.createData()

# get joint position ranges
# q_max = robot.model.upperPositionLimit.T
# q_min = robot.model.lowerPositionLimit.T
q_min = np.array([-np.pi, -2.9, -np.pi, -2.22,-np.pi/4,-np.pi/4,-np.pi])
# q_max = np.array([np.pi, 0.55, np.pi, 0.02,np.pi/4,np.pi/4,np.pi])
q_max = np.array([np.pi, 0, np.pi, 0.0,np.pi/4,np.pi/4,np.pi])
robot.model.upperPositionLimit = q_max
robot.model.lowerPositionLimit = q_min
dq_max = np.ones(robot.nq)
dq_min = -dq_max
# get max velocity
t_max = np.ones(robot.nq)*4 # amps
t_min = -t_max

# Use robot configuration.
# q0 = np.random.uniform(q_min,q_max)
q = (q_min+q_max)/2

viz = MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
# Start a new MeshCat server and client.
viz.initViewer(open=True)
# Load the robot in the viewer.
viz.loadViewerModel()
viz.display(q)
# small time window for loading the model 
# if meshcat does not visualise the robot properly, augment the time
# it can be removed in most cases
time.sleep(0.2) 

viz.viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7005/static/


In [4]:
from ipywidgets import interact, FloatSlider
kwargs = {'q[{}]'.format(i) : 
          FloatSlider(
              min = q_min[i], 
              max = q_max[i], 
              step = 0.01, 
              value = q[i]) 
          for i,q_1 in enumerate(q)}
@interact(**kwargs)
def update(**kwargs):
    q = np.array([v  for v in kwargs.values()])
    viz.display(q)


interactive(children=(FloatSlider(value=0.0, description='q[0]', max=3.141592653589793, min=-3.141592653589793…

In [ ]:
import meshcat_shapes

# import meshcat_shapes
import numpy as np
import qpsolvers

import pink
from pink import solve_ik
from pink.tasks import FrameTask, PostureTask
from pink.utils import custom_configuration_vector
from pink.visualization import start_meshcat_visualizer

end_effector_task = FrameTask(
    "r_arm_tip",
    position_cost=1.0,  # [cost] / [m]
    orientation_cost=1.0,  # [cost] / [rad]
)

posture_task = PostureTask(
    cost=1e-2,  # [cost] / [rad]
)
posture_task.set_target((q_min+q_max)/2)

tasks = [end_effector_task, posture_task]

configuration = pink.Configuration(robot.model, robot.data, q)
for task in tasks:
    task.set_target_from_configuration(configuration)

viewer = viz.viewer
meshcat_shapes.frame(viewer["end_effector_target"], opacity=0.5)

# Select QP solver
solver = qpsolvers.available_solvers[0]
if "quadprog" in qpsolvers.available_solvers:
    solver = "quadprog"
# rate = RateLimiter(frequency=200.0)
dt = 0.01
t = 0.0  # [s]
while True:
    # Update task targets
    end_effector_target = end_effector_task.transform_target_to_world
    end_effector_target.rotation = -np.eye(3)[:,[1,2,0]]
    end_effector_target.translation = np.array([0.4,-0.15+0.1*np.cos(t),0.8+0.2*np.sin(t)])

    # Update visualization frames
    viewer["end_effector_target"].set_transform(end_effector_target.np)

    # Compute velocity and integrate it into next configuration
    velocity = solve_ik(configuration, tasks, dt, solver=solver)
    configuration.integrate_inplace(velocity, dt)

    # Visualize result at fixed FPS
    viz.display(configuration.q)
    # rate.sleep()
    t += dt
